In [82]:
# -----------------------
# depencies
# -----------------------
import json
import os
import uuid
import random
import sys
from IPython.display import clear_output
from math import trunc

# -----------------------
# load settings
# -----------------------
sys.path.append('./data/')
from data import settings

# -----------------------
# SYSTEM functions 
# -----------------------
def criar_transacoes(num_transacoes, proporcao_categorias, seed=settings.seed):
    assert sum([proporcao_categorias[k] for k in proporcao_categorias])==1, '`proporcao_categorias` não soma 100%! Favor rever.'

    # garantir reprodutibilidade dos valores
    random.seed(seed)

    # Calcula o número de transações por categoria com base na proporção
    numero_transacoes_por_categoria = {categoria: int(num_transacoes * proporcao) for categoria, proporcao in proporcao_categorias.items()}
    
    transacoes = []
    
    # Gera as transações
    for categoria, quantidade in numero_transacoes_por_categoria.items():
        for _ in range(quantidade):
            transacao = {
                "UUID": str(uuid.uuid4()),
                "valor": round(random.uniform(1.0, 1000.0), 2),  # Preço aleatório entre 1 e 1000
                "categoria": categoria
            }
            transacoes.append(transacao)
    
    return transacoes

def salvar_json(transacoes, path2save, filename):
    # create path if not exist
    if not os.path.exists(path2save):
        os.makedirs(path2save)
    with open(os.path.join(path2save,filename), "w") as file:
        json.dump(transacoes, file, indent=4)
    print(f"Arquivo salvo em: {os.path.abspath(os.path.curdir)+'/'+path2save+'/'+filename}")

def criar_bd(num_transacoes:int = 10000, proporcao_categorias:list = settings.categorias_proporcao, path2save="./data", filename='transactions.json'):
    salvar_json(criar_transacoes(num_transacoes,  proporcao_categorias),
                path2save, filename
    )

def load_bd(filepath='./data/transactions.json'):
    with open(filepath, "r") as file:
        bd = json.load(file)
    return bd

def tela_inicial(usuario, conta):
    print(f"Bem-vindo, {usuario}!")
    print(f"Conta: {conta}")
    print("\nEste programa permite gerenciar transações de sua conta pessoal.")
    print("\nEscolha uma das opções abaixo:")
    print("1. Visualizar relatórios")
    print("2. Cadastrar transações")
    print("3. Editar transações")
    print("4. Excluir transações")
    print("-" * 10)
    print("0. Sair")
    print('\n')

In [83]:
def validar_opcao_sim_nao(pergunta):
    while True:
        confirmacao = input(f"{pergunta} \n1: Sim\n2: Não").strip()
        if confirmacao == "1" or confirmacao == "2":
            clear_output(wait = True)
            break
        else:
            print("Opção inválida. ** Digite 1 para Sim e 2 para Não **")
            clear_output(wait = True)
    return confirmacao


def consultar_transacao_por_ID(id, bd):
    try:
        transacao = [transacao for transacao in bd if transacao["UUID"] == id][0]
        clear_output(wait = True)
        return transacao
    except IndexError:
        print("O ID informado não foi encontrado. Por favor, digite novamente.")
        clear_output(wait = True)
    except:
        print("Erro inesperado. Tente novamente.")
        clear_output(wait = True)
    return None


def validar_valor(valor):
    try:
        valor = float(valor)
        if valor == 0:
            raise Exception("O valor não pode ser zero.")
        elif valor < 0:
            raise Exception("O valor não pode ser negativo.")
        clear_output(wait = True)
        return round(valor, 2)
    except ValueError:
        print("O valor inserido não é válido. ** Use apenas números e ponto como separador decimal **")
        clear_output(wait = True)
    except Exception as erro:
        print(erro)
        clear_output(wait = True)
    except:
        print("Valor inválido.")
        clear_output(wait = True)
    return None


def validar_categoria(categoria):
    dict_categoria = {
        "1": "casa",
        "2": "lazer",
        "3": "viagens",
        "4": "investimentos",
        "5": "transferencias",
        "6": "saude",
        "7": "alimentacao"
        }
    try:
        categoria = dict_categoria[categoria]
        clear_output(wait = True)
        return categoria
    except KeyError:
        print("A categoria não existe. ** Digite um número entre 1 e 7 **")
        clear_output(wait = True)
    except:
        print("Opção inválida.")
        clear_output(wait = True)
    return None


def cadastrar_transacao(valor, categoria, bd):
    transacao = {
        "UUID": str(uuid.uuid4()),
        "valor": valor,
        "categoria": categoria
    }
    bd.append(transacao)
    return salvar_json(bd, "./data", "transactions.json")


def editar_transacao_por_ID(transacao, opcao, bd):
    if opcao == "1":
        while True:
            novo_valor = input("Digite o novo valor da transação: ").strip()
            novo_valor_validado = validar_valor(novo_valor)
            if novo_valor_validado is not None:
                transacao["valor"] = novo_valor_validado
                break
    elif opcao == "2":
        while True:
            nova_categoria = input("Escolha a nova categoria da transação: \n1: casa\n2: lazer\n3: viagens\n4: investimentos\n5: transferencias\n6: saude\n7: alimentacao").strip()
            nova_categoria_validada = validar_categoria(nova_categoria)
            if nova_categoria_validada is not None:
                transacao["categoria"] = nova_categoria_validada
                break
    else:
        while True:
            novo_valor = input("Digite o novo valor da transação: ").strip()
            novo_valor_validado = validar_valor(novo_valor)
            if novo_valor_validado is not None:
                transacao["valor"] = novo_valor_validado
                break
        while True:
            nova_categoria = input("Escolha a nova categoria da transação: \n1: casa\n2: lazer\n3: viagens\n4: investimentos\n5: transferencias\n6: saude\n7: alimentacao").strip()
            nova_categoria_validada = validar_categoria(nova_categoria)
            if nova_categoria_validada is not None:
                transacao["categoria"] = nova_categoria_validada
                break
    print(f"Confira os dados atualizados: \nUUID: {transacao["UUID"]}\nValor: R$ {transacao['valor']:.2f}\nCategoria: {transacao['categoria']}")
    for dict_transacao in bd:
        if dict_transacao["UUID"] == transacao["UUID"]:
            dict_transacao["valor"] = transacao["valor"]
            dict_transacao["categoria"] = transacao["categoria"]
    with open("./data/transactions.json", "w", encoding = "utf-8") as json_file:
        json.dump(bd, json_file, indent = 4, ensure_ascii = False)
    print("Transação editada com sucesso!")


def excluir_transacao(id, bd):
    bd = [transacao for transacao in bd if transacao["UUID"] != id]

    with open("./data/transactions.json", "w", encoding = "utf-8") as json_file:
        json.dump(bd, json_file, indent = 4, ensure_ascii = False)
    print("Transação excluída com sucesso!")


def relatorio_por_categoria(bd, operacao):
    lista_categorias = ["casa", "lazer", "viagens", "investimentos", "transferencias", "saude", "alimentacao"]
    dict_relatorios_categorias = {}
    for categoria in lista_categorias:
        filtro_categoria = list(filter(lambda x: x.get("categoria") == categoria, bd))
        if operacao == "total":
            if len(filtro_categoria) > 0:
                total = round(sum([transacao["valor"] for transacao in filtro_categoria]), 2)
                dict_relatorios_categorias[categoria] = total
        elif operacao == "media":
            if len(filtro_categoria) > 0:
                media = round(sum([transacao["valor"] for transacao in filtro_categoria]) / len(filtro_categoria), 2)
                dict_relatorios_categorias[categoria] = media
        elif operacao == "maiores":
            if len(filtro_categoria) > 0:
                cinco_maiores_transacoes = sorted(filtro_categoria, key = lambda transacao: transacao["valor"], reverse = True)[:5]
                dict_relatorios_categorias[categoria] = cinco_maiores_transacoes
        elif operacao == "menores":
            if len(filtro_categoria) > 0:
                cinco_menores_transacoes = sorted(filtro_categoria, key = lambda transacao: transacao["valor"])[:5]
                dict_relatorios_categorias[categoria] = cinco_menores_transacoes
        elif operacao == "medianas":
            if len(filtro_categoria) > 0:
                transacoes_ordenadas = sorted(filtro_categoria, key = lambda transacao: transacao["valor"])
                centro_da_lista = len(transacoes_ordenadas) / 2
                if len(transacoes_ordenadas) < 5:
                    cinco_transacoes_medianas = transacoes_ordenadas
                elif len(transacoes_ordenadas) % 2 == 0:
                    cinco_transacoes_medianas = transacoes_ordenadas[(int(centro_da_lista) - 2) : (int(centro_da_lista) + 3)]
                else:
                    cinco_transacoes_medianas = transacoes_ordenadas[(trunc(centro_da_lista) - 2) : (round(centro_da_lista) + 2)]
                dict_relatorios_categorias[categoria] = cinco_transacoes_medianas
    return dict_relatorios_categorias


def calcular_total_transacoes(bd, categoria):
    total_transacoes = sum([transacao["valor"] for transacao in bd])
    if categoria == "1":
        totais_por_categoria = relatorio_por_categoria(bd, "total")
    else:
        totais_por_categoria = None
    return total_transacoes, totais_por_categoria


def mostrar_m5_transacoes(bd, categoria, operacao):
    if operacao == "maiores":
        m5_transacoes = sorted(bd, key = lambda transacao: transacao["valor"], reverse = True)[:5]
    elif operacao == "menores":
        m5_transacoes = sorted(bd, key = lambda transacao: transacao["valor"])[:5]
    elif operacao == "medianas":
        transacoes_ordenadas = sorted(bd, key = lambda transacao: transacao["valor"])
        centro_da_lista = len(transacoes_ordenadas) / 2
        if len(transacoes_ordenadas) < 5:
            m5_transacoes = transacoes_ordenadas
        elif len(transacoes_ordenadas) % 2 == 0:
            m5_transacoes = transacoes_ordenadas[(int(centro_da_lista) - 2) : (int(centro_da_lista) + 3)]
        else:
            m5_transacoes = transacoes_ordenadas[(trunc(centro_da_lista) - 2) : (round(centro_da_lista) + 2)]
    if categoria == "1":
        m5_por_categoria = relatorio_por_categoria(bd, operacao)
    else:
        m5_por_categoria = None
    return m5_transacoes, m5_por_categoria


def calcular_media(bd, categoria):
    media_transacoes = sum([transacao["valor"] for transacao in bd]) / len(bd)
    if categoria == "1":
        medias_por_categoria = relatorio_por_categoria(bd, "media")
    else:
        medias_por_categoria = None
    return media_transacoes, medias_por_categoria


def salvar_relatorio(relatorio, nome_relatorio, diretorio):
    with open(f"{diretorio}/{nome_relatorio}", "w", encoding = "utf-8") as arquivo:
        if type(relatorio) == list:
            for transacao in relatorio:
                linha = transacao + "\n\n"
                arquivo.write(linha)
        else:
            arquivo.write(relatorio)
    print(f"Relatório salvo em {diretorio}/{nome_relatorio}")


def processar_relatorio(operacao , diretorio):
    categoria = validar_opcao_sim_nao(f"Deseja calcular {operacao.replace("e", "é") if operacao == "media" else operacao} por categoria também?")
    if operacao == "total":
        total_transacoes, por_categorias = calcular_total_transacoes(bd, categoria)
        relatorio = f"O total das transações é R$ {total_transacoes:.2f}"
    elif operacao == "media":
        media_transacoes, por_categorias = calcular_media(bd, categoria)
        relatorio = f"A média das transações é R$ {media_transacoes:.2f}"
    print(relatorio)
    salvar_relatorio(relatorio, f"{operacao}_transacoes.txt", diretorio)
    if por_categorias is not None:
        relatorio_categorias = []
        for categoria in por_categorias:
            relatorio_categoria = f"{operacao.capitalize().replace("e", "é") if operacao == "media" else operacao.capitalize()} da categoria '{categoria}': R$ {por_categorias.get(categoria):.2f}"
            relatorio_categorias.append(relatorio_categoria)
            print(relatorio_categoria) 
        salvar_relatorio(relatorio_categorias, f"{operacao}_por_categoria.txt", diretorio)


def processar_relatorio_m5(operacao, diretorio):
    categoria = validar_opcao_sim_nao(f"Deseja calcular top 5 {operacao} por categoria também?")
    m5_transacoes, m5_por_categoria = mostrar_m5_transacoes(bd, categoria, operacao)
    relatorio = f"As top 5 {operacao} são:\n"
    for indice, transacao in enumerate(m5_transacoes):
        relatorio += f"({indice + 1}) \n\tUUID: {transacao["UUID"]}\n\tValor: R$ {transacao["valor"]:.2f}\n\tCategoria: {transacao['categoria']}\n"
    print(relatorio)
    salvar_relatorio(relatorio, f"top5_{operacao}.txt", diretorio)
    if m5_por_categoria is not None:
        relatorio_categorias = []
        for categoria in m5_por_categoria:
            relatorio_categoria = f"As top 5 {operacao} da categoria '{categoria}' são:"
            for indice, transacao in enumerate(m5_por_categoria[categoria]):
                relatorio_categoria += f"\n({indice + 1}) \n\tUUID: {transacao["UUID"]}\n\tValor: R$ {transacao["valor"]:.2f}"
            relatorio_categorias.append(relatorio_categoria)
            print(relatorio_categoria)
            print()
        salvar_relatorio(relatorio_categorias, f"top5_{operacao}_por_categoria.txt", diretorio)


def visualizar_relatorios():
    print("\nEste menu permite visualizar os relatórios de sua conta pessoal.")
    print("\nEscolha uma das opções abaixo:")
    print("1. Total das transações")
    print("2. Top 5 maiores transações")
    print("3. Top 5 menores transações")
    print("4. Top 5 transações medianas")
    print("5. Média transações")
    print("-" * 10)
    print("0. Sair")
    print('\n')
    opcao_escolhida = input().strip()
    clear_output(wait = True)
    while True:
        if opcao_escolhida == "0":
            print("SAINDO...")
            break
        elif opcao_escolhida == "1":
            print("*** TOTAL DAS TRANSAÇÕES ***")
            processar_relatorio("total", "./reports")
            break
        elif opcao_escolhida == "2":
            print("*** TOP 5 MAIORES TRANSAÇÕES ***")
            processar_relatorio_m5("maiores", "./reports")
            break
        elif opcao_escolhida == "3":
            print("*** TOP 5 MENORES TRANSAÇÕES ***")
            processar_relatorio_m5("menores", "./reports")
            break
        elif opcao_escolhida == "4":
            print("*** TOP 5 TRANSAÇÕES MEDIANAS ***")
            processar_relatorio_m5("medianas", "./reports")
            break
        elif opcao_escolhida == "5":
            print("*** MÉDIA DAS TRANSAÇÕES ***")
            processar_relatorio("media", "./reports")
            break


def run():
    usuario = input('Olá! Vamos iniciar seu atendimento. Por favor, insira seu nome:').title().strip()
    conta = input('Insira o número da sua conta:').strip()

    opcao_escolhida = None

    while True:

        tela_inicial(usuario, conta)
        opcao_escolhida = input().strip()
        clear_output(wait = True)
        
        if opcao_escolhida == "0":
            print("SAINDO...")
            break


        elif opcao_escolhida == "1":
            print("*** VISUALIZAR RELATÓRIOS ***")
            continuar = validar_opcao_sim_nao("Deseja continuar?")
            if continuar == "2":
                clear_output()
                continue
            while True:
                visualizar_relatorios()
                
                visualizar_outros_relatorios = validar_opcao_sim_nao("Deseja visualizar outro relatório?")
                if visualizar_outros_relatorios == "2":
                    break


        elif opcao_escolhida == "2":
            print("*** CADASTRAR TRANSAÇÃO ***")
            continuar = validar_opcao_sim_nao("Deseja continuar?")
            if continuar == "2":
                clear_output()
                continue
            while True:
                while True:
                    valor = input("Informe o valor da transação: ").strip()
                    valor_validado = validar_valor(valor)
                    if valor_validado is not None:
                        break
                while True:    
                    categoria = input("Escolha a categoria da transação: \n1: casa\n2: lazer\n3: viagens\n4: investimentos\n5: transferencias\n6: saude\n7: alimentacao").strip()
                    categoria_validada = validar_categoria(categoria)
                    if categoria_validada is not None:
                        break
                print(f"Valor: R$ {valor_validado:.2f}\nCategoria: {categoria_validada}")

                cadastrar_transacao(valor_validado, categoria_validada, bd)
                print("Transação cadastrada com sucesso!")
                clear_output(wait = True)

                cadastrar_mais_transacoes = validar_opcao_sim_nao("Deseja cadastrar outra transação?")
                if cadastrar_mais_transacoes == "2":
                    break

        
        elif opcao_escolhida == "3":
            print("*** EDITAR TRANSAÇÃO ***")
            continuar = validar_opcao_sim_nao("Deseja continuar?")
            if continuar == "2":
                clear_output()
                continue
            while True:
                while True:
                    id = input("Informe o ID da transação que gostaria de editar: ").lower().strip()
                    transacao = consultar_transacao_por_ID(id, bd)
                    if transacao is not None:
                        break
                print(f"A transação selecionada foi: \nUUID: {transacao["UUID"]}\nValor: R$ {transacao["valor"]:.2f}\nCategoria: {transacao['categoria']}")
                clear_output(wait = True)
                while True:
                    escolher_opcao_editar = input("Qual informação você gostaria de editar? \n1: Valor\n2: Categoria\n3: Ambos").strip()
                    if escolher_opcao_editar in ["1", "2", "3"]:
                        editar_transacao_por_ID(transacao, escolher_opcao_editar, bd)
                        clear_output(wait = True)
                        break
                    else:
                        print("Opção inválida. Tente novamente.")
                        clear_output(wait = True)
                        continue
                
                editar_mais_transacoes = validar_opcao_sim_nao("Deseja editar outra transação?")
                if editar_mais_transacoes == "2":
                    break
        

        elif opcao_escolhida == "4":
            print("*** EXCLUIR TRANSAÇÃO ***")
            continuar = validar_opcao_sim_nao("Deseja continuar?")
            if continuar == "2":
                clear_output()
                continue
            while True:
                while True:
                    id = input("Informe o ID da transação que gostaria de excluir: ").lower().strip()
                    transacao = consultar_transacao_por_ID(id, bd)
                    if transacao is not None:
                        break
                print(f"A transação selecionada foi: \nUUID: {transacao["UUID"]}\nValor: R$ {transacao["valor"]:.2f}\nCategoria: {transacao['categoria']}")
                clear_output(wait = True)

                confirmar_exclusao = validar_opcao_sim_nao("Deseja prosseguir com a exclusão?")
                if confirmar_exclusao == "2":
                    clear_output()
                    break

                excluir_transacao(transacao["UUID"], bd)
                clear_output(wait = True)

                excluir_mais_transacoes = validar_opcao_sim_nao("Deseja excluir outra transação?")
                if excluir_mais_transacoes == "2":
                    break
        
        
        else:
            print("Opção inválida, digite novamente.")

In [ ]:

# -----------------------
# MAIN SCRIPT
# -----------------------
if __name__ == "__main__":
    
    # criar o banco de dados caso ele não exista
    print(os.path.abspath('.'))
    if not os.path.exists('./data/transactions.json'):
        criar_bd()
    
    # load bd 
    bd = load_bd()
   
    # inicia o programa
    run()